In [1]:
#import libraries
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras import backend as K
import tensorflow as tf

import numpy as np
import pandas as pd
import os
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf 
from statsmodels.tsa.seasonal import seasonal_decompose 

from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline


tf.compat.v1.enable_eager_execution()

Using TensorFlow backend.


In [ ]:
import pandas as pd
#import data

#import main data
folder = r'\\smc-filer\divsede\APPS\EVA\notebooks\notebooks\users\Hani_Yacoub\\'
name = 'data2021.csv'
df = pd.read_csv(folder + name)
df = df.drop(["Unnamed: 0"],axis=1)

branches = [9,12,1,10,2]
products = [31,21,31,48,1,29]

df = df[df['branch_code'].isin(branches)]
df = df[df['item_cat'].isin(products)]

df = df.reset_index().drop(["index"],axis=1)

#create data for all, and just for 2019 to test it

df_2019 = df[df["year"]==2019]

df = df[df["year"]!=2019]


# make a prediction with a stacking ensemble
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor


X = df.drop(["sales","ym"],axis=1)

y = df[["sales"]]

from sklearn.model_selection import KFold, cross_val_score, train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_test_orginal = X_test

X_train = pd.get_dummies(X_train, columns=['branch_code', 'item_cat', 'promotion', 'm'])
X_test = pd.get_dummies(X_test, columns=['branch_code', 'item_cat', 'promotion', 'm'])

# define the base models
level0 = list()
level0.append(('knn', KNeighborsRegressor(n_neighbors=10)))
level0.append(('cart', DecisionTreeRegressor()))
level0.append(('RF', RandomForestRegressor(n_estimators=100,max_features='log2', verbose=1)))
level0.append(('svm', SVR()))
level0.append(('nn', MLPRegressor(random_state=1, max_iter=500)))

# define meta learner model
level1 = LinearRegression()
# define the stacking ensemble
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
# fit the model on all available data
model.fit(X_train, y_train)

# make a prediction for one example
yhat = model.predict(X_test)

from sklearn.metrics import r2_score

print(r2_score(y_test, yhat))

#we need to see how our model did on unseen data that is in the future, to see its forcasting power

#we take the validation dataset which is in the year 2019, and we take the month of Feburary

feb = df_2019[df_2019["m"]==2]

#keep the oringal dataset to make it easz to tranform it back from the get_dummies
feb_or = feb

#encode the categorlical data
feb = pd.get_dummies(feb, columns=['branch_code', 'item_cat', 'promotion', 'm'])

#drop uneeded columns
feb2 = feb.drop(["ym","sales"],axis=1)

#enter missing columns
for i in list(set(X_test.columns).difference(feb2.columns)):
    
    feb2[i] = 0

#predict it using our Stacking model
yhat = model.predict(feb2)

#add our prediction to the dataset of Feburary
feb_or["yhat"] = yhat

#lets see how good our model did
print(r2_score(feb_or["sales"], feb_or["yhat"]))

#lets plot the results. (pay attention the graph to understand it well, each point is an obserbvation in the month of Feb)
feb_or.groupby(["branch_code","item_cat","ym"]).sum().plot(figsize=(20,20))


#now we need to add the prediction of LSTM and the predictions of FACEBOOK prophet and see if we can get a better reuslts
fbpred = []
fbpred1 = []
fbpred2 = []

import pandas as pd
from fbprophet import Prophet


for i in list(df["branch_code"].unique()):
    for y in list(df["item_cat"].unique()):
        
        x = df[(df["branch_code"]==i)&(df["item_cat"]==y)][["ym","sales"]].groupby("ym").sum().reset_index().rename({"ym":"ds","sales":"y"},axis=1)
        
        m = Prophet(seasonality_mode='multiplicative').fit(x)
        future = m.make_future_dataframe(periods=3652)
        
        m = Prophet(seasonality_mode='multiplicative', mcmc_samples=300).fit(x)
        fcst = m.predict(future)

        future = m.make_future_dataframe(periods=120, freq='MS')
        fcst = m.predict(future)
        
        num = max(0, int(fcst.set_index("ds")[71:72]["yhat"]))
        brnach_code1 = i
        product_id = y
        
        
        
        fbpred.append(num)
        fbpred1.append(brnach_code1)
        fbpred2.append(product_id)
        
fp = pd.DataFrame()

fp["pred"] = fbpred
fp["branch"] = fbpred1
fp["product"] = fbpred2

fp

final1 = pd.merge(feb_or, fp,  how='left', left_on=['branch_code','item_cat'], right_on = ['branch','product'])
final1.groupby(["branch_code","item_cat","ym"]).sum().plot(figsize=(20,20))


#LSTM

import pandas as pd
#import data

#import main data
folder = r'\\smc-filer\divsede\APPS\EVA\notebooks\notebooks\users\Hani_Yacoub\\'
name = 'data2021.csv'
df = pd.read_csv(folder + name)
df = df.drop(["Unnamed: 0"],axis=1)

branches = [9,12,1,10,2]
products = [31,21,31,48,1,29]

df = df[df['branch_code'].isin(branches)]
df = df[df['item_cat'].isin(products)]

df = df.reset_index().drop(["index"],axis=1)

x = df[(df["branch_code"]==2)&(df["item_cat"]==21)][["ym","sales"]].groupby("ym").sum().reset_index().rename({"ym":"ds","sales":"y"},axis=1)

x = x.set_index("ds")
x.index.freq = 'MS'


#now we need to add the prediction of LSTM and the predictions of FACEBOOK prophet and see if we can get a better reuslts
fbpred = []
fbpred1 = []
fbpred2 = []

fp = pd.DataFrame()

import pandas as pd
from fbprophet import Prophet


for i in list(df["branch_code"].unique()):
    for y in list(df["item_cat"].unique()):
        
        print(i)
        print(y)
        
        x = df[(df["branch_code"]==i)&(df["item_cat"]==y)][["ym","sales"]].groupby("ym").sum().reset_index().rename({"ym":"ds","sales":"y"},axis=1)
        
        x = x.set_index("ds")
        # fix random seed for reproducibility
        seed = numpy.random.seed(7)

        # normalize the dataset
        data_range = (-1, 1)
        scaler = MinMaxScaler(feature_range=data_range)        # scaler can also de-normalize the dataset by scaler.inverse_transform(), useful for actual prediction
        dataset_scaled = scaler.fit_transform(x)
        #dataset_scaled = numpy.array(dataset_scaled)

        # split into train and test sets
        train_size = int(len(dataset_scaled) * 0.67)
        test_size = len(dataset_scaled) - train_size
        train, test = dataset_scaled[0:train_size,:], dataset_scaled[train_size:len(x),:]
        print(len(train), len(test))

        # convert an array of values into a dataset matrix
        def create_dataset(data, look_back=1):
            dataX, dataY = [], []
            i_range = len(data) - look_back - 1
            print(i_range)
            for i in range(0, i_range):
                dataX.append(data[i:(i+look_back)])    # index can move down to len(dataset)-1
                dataY.append(data[i + look_back])      # Y is the item that skips look_back number of items

            return numpy.array(dataX), numpy.array(dataY)

        # try it
        look_back = 4
        dataX, dataY = create_dataset(dataset_scaled, look_back=look_back)

        print("X shape:", dataX.shape)
        print("Y shape:", dataY.shape)

        print("Xt-3     Xt-2      Xt-1      Xt        Y")
        print("---------------------------------------------")
        for z in range(len(dataX)): 
            print('%.2f   %.2f    %.2f    %.2f    %.2f' % (dataX[z][0][0], dataX[z][1][0], dataX[z][2][0], dataX[z][3][0], dataY[z][0]))


        # Reshape to (samples, timestep, features)
        dataX = numpy.reshape(dataX, (dataX.shape[0], 1, dataX.shape[1]))

        print("X shape:", dataX.shape)

        # reshape into X=t and Y=t+1
        look_back = 1
        trainX, trainY = create_dataset(train, look_back)      # trainX is input, trainY is expected output
        testX, testY = create_dataset(test, look_back)


        # reshape input to be [samples, time steps, features]
        print("Original trainX shape:", trainX.shape)
        trainX = numpy.reshape(trainX, (trainX.shape[0], look_back, trainX.shape[1]))     # timestep = 1, input_dim = trainX.shape[1]
        testX = numpy.reshape(testX, (testX.shape[0], look_back, testX.shape[1]))
        print("New trainX shape:", trainX.shape)
        print("trainY shape:", trainY.shape)
        print("trainY example:", trainY[0])


        # create and fit the LSTM network
        from keras.layers import Dropout

        batch_size = 1
        timesteps = trainX.shape[1]
        input_dim = trainX.shape[2]

        model = Sequential()
        #model.add(LSTM(8, input_shape=(1, look_back)))    # 4 or 8 is the number of LSTM units or dimensions of output of LSTM layer, can be any integer. input_shape must show up in the first layer
        model.add(LSTM(4, batch_input_shape=(batch_size, timesteps, input_dim)))
        model.add(Dense(10))    # not necessary, but just want to try it
        #model.add(Dropout(0.8))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(trainX, trainY, epochs=50, batch_size=1, verbose=2)   # it turns out that epochs = 15 is enough, 100 is not necessary


        # make predictions
        trainPredict = model.predict(trainX, batch_size)
        testPredict = model.predict(testX, batch_size)      

        # invert predictions
        trainPredict = scaler.inverse_transform(trainPredict)
        trainY = scaler.inverse_transform(trainY)               # trainY is of shape (samples, features) while trainX is of (samples, timesteps, features) )
        testPredict = scaler.inverse_transform(testPredict)
        testY = scaler.inverse_transform(testY)

        print("trainY shape:", trainY.shape)
        print("trainPredict shape:", trainPredict.shape)
        print("testY shape:", testY.shape)
        print("testPredict shape:", testPredict.shape)

        # calculate root mean squared error
        trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
        print('Train Score: %.2f RMSE' % (trainScore))
        testScore = math.sqrt(mean_squared_error(testY, testPredict))
        print('Test Score: %.2f RMSE' % (testScore))


        # shift train predictions for plotting
        trainPredictPlot = numpy.empty_like(x)
        trainPredictPlot[:, :] = numpy.nan
        #print(trainPredictPlot[0])
        trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

        # shift test predictions for plotting
        testPredictPlot = numpy.empty_like(x)
        testPredictPlot[:, :] = numpy.nan
        testPredictPlot[len(trainPredict)+(look_back*2)+1:len(x)-1, :] = testPredict

        # plot baseline and predictions
        plt.plot(scaler.inverse_transform(dataset_scaled))
        #plt.plot(dataset_scaled)
        plt.plot(trainPredictPlot)
        plt.plot(testPredictPlot)
        plt.show()

        x = x.reset_index()
        x["test"] = testPredictPlot
        
        if len(x[x["ds"]=="2019-02"]["test"])!=0:
            num = int(x[x["ds"]=="2019-02"]["test"])
            brnach_code1 = i
            product_id = y

            x = df[(df["branch_code"]==i)&(df["item_cat"]==y)][["ym","sales"]].groupby("ym").sum().reset_index().rename({"ym":"ds","sales":"y"},axis=1)





            fbpred.append(num)
            fbpred1.append(brnach_code1)
            fbpred2.append(product_id)
        

fp["pred"] = fbpred
fp["branch"] = fbpred1
fp["product"] = fbpred2


final2 = pd.merge(final1, fp,  how='left', left_on=['branch_code','item_cat'], right_on = ['branch','product'])
final2.groupby(["branch_code","item_cat","ym"]).sum().plot(figsize=(20,20))

final3 = final2[["branch_code","item_cat","ym","yhat","pred_x","pred_y","sales"]].groupby(["branch_code","item_cat","ym"]).sum().reset_index()

final3["end"] = (final3["yhat"] + final3["pred_x"] + final3["pred_y"] )/3

final3.groupby(["branch_code","item_cat","ym"]).sum()[["sales","end"]].plot(figsize=(20,20))

print('Stacking = ',r2_score(final3["sales"],final3["yhat"]))
print('FP = ',r2_score(final3["sales"],final3["pred_x"]))
print('LSTM = ',r2_score(final3["sales"],final3["pred_y"]))
print('AVG = ',r2_score(final3["sales"],final3["end"]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j